<a href="https://colab.research.google.com/github/AkseliManninen/A-Star-Search-Algorithm-Visualized/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing and importing libraries

In [33]:
# Importing libraries
import os
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.utils.data as data

from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Configuration

In [9]:
# Select the device for training (use GPU if you have one)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.is_available():
  print("Using a GPU")
else:
  print("Using a CPU")

Using a CPU


In [10]:
# Changing the working directory
from google.colab import drive
drive.mount('/content/drive')

# Path to the "Tutkimusprojekti" folder in the root of Google Drive
google_drive_path = '/content/drive/My Drive/Opinnot/Diplomityö/Koodi/dippa/models'

# Change the current directory to the "Tutkimusprojekti" folder
os.chdir(google_drive_path)

# Check the current working directory to verify the change
data_dir = os.getcwd()
print("Current working directory:", data_dir)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current working directory: /content/drive/My Drive/Opinnot/Diplomityö/Koodi/dippa/models


In [16]:
# Define the grid size to be either 1000 or 250
grid_size = 1000

df = pd.read_csv(f'/content/dataset_{grid_size}.csv')

#df["Count"] = df["Count"].astype(float)
#df["Kuukauden sadesumma [mm]"] = df["Kuukauden sadesumma [mm]"].astype(float)
#df["Kuukauden keskilämpötila [°C]"] = df["Kuukauden keskilämpötila [°C]"].astype(float)

#column_to_move = df.pop("Count")
#df.insert(df.shape[1], "Count", column_to_move)

# Define the input size based on the number of features
input_size = df.shape[1] - 2
df.head()

,grid_1000,Month_Year,RAIN,TEMP,X,Y,DIST,Q1,Q2,Q3,Q4,PAVED,DIRT,R,G,B,COUNT
0,0.0,2014-01-01,49.9,-6.4,389000.0,6661000.0,11726.8,1,0,0,0,0,0,24.013605,36.812925,45.510204,0.0
1,0.0,2014-02-01,29.4,-0.1,389000.0,6661000.0,11726.8,1,0,0,0,0,0,24.013605,36.812925,45.510204,0.0
2,0.0,2014-03-01,32.2,2.0,389000.0,6661000.0,11726.8,1,0,0,0,0,0,24.013605,36.812925,45.510204,0.0
3,0.0,2014-04-01,9.9,6.0,389000.0,6661000.0,11726.8,0,1,0,0,0,0,24.013605,36.812925,45.510204,0.0
4,0.0,2014-05-01,59.1,10.8,389000.0,6661000.0,11726.8,0,1,0,0,0,0,24.013605,36.812925,45.510204,0.0


In [17]:
# train-test split for time series
train = df[df["Month_Year"].str.contains("2022") == False]
train.reset_index(drop=True, inplace=True)

test = df[df["Month_Year"].str.contains("2022")]
test.reset_index(drop=True, inplace=True)

test.head()

,grid_1000,Month_Year,RAIN,TEMP,X,Y,DIST,Q1,Q2,Q3,Q4,PAVED,DIRT,R,G,B,COUNT
0,0.0,2022-01-01,102.3,-2.6,389000.0,6661000.0,11726.8,1,0,0,0,0,0,24.013605,36.812925,45.510204,0.0
1,0.0,2022-02-01,127.5,-1.4,389000.0,6661000.0,11726.8,1,0,0,0,0,0,24.013605,36.812925,45.510204,0.0
2,0.0,2022-03-01,14.9,0.8,389000.0,6661000.0,11726.8,1,0,0,0,0,0,24.013605,36.812925,45.510204,0.0
3,0.0,2022-04-01,34.7,4.1,389000.0,6661000.0,11726.8,0,1,0,0,0,0,24.013605,36.812925,45.510204,0.0
4,0.0,2022-05-01,50.7,9.8,389000.0,6661000.0,11726.8,0,1,0,0,0,0,24.013605,36.812925,45.510204,0.0


In [19]:
def create_dataset(dataset, lookback):
    """Transform a time series into a prediction dataset

    Args:
        dataset: A numpy array of time series, first dimension is the time steps
        lookback: Size of window for prediction
    """
    X_batches, y_batches = [], []
    batch_size = 64  # Adjust batch size according to your available memory
    for i in range(0, len(dataset) - lookback, batch_size):
        batch_X, batch_y = [], []
        for j in range(i, min(i + batch_size, len(dataset) - lookback)):
            if dataset[f"grid_{grid_size}"].iloc[j] == dataset[f"grid_{grid_size}"].iloc[j + lookback]:
                feature = dataset.iloc[:, 2:].iloc[j:j + lookback].to_numpy()
                target = dataset["COUNT"].iloc[j + lookback]
                batch_X.append(feature)
                batch_y.append(target)
        X_batches.extend(torch.tensor(batch_X).float())
        y_batches.extend(torch.tensor(batch_y).float())
    return X_batches, y_batches

lookback = 8
X_train_batches, y_train_batches = create_dataset(train, lookback=lookback)
X_test_batches, y_test_batches = create_dataset(test, lookback=lookback)


<ipython-input-19-0fd89d5159b9>:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X_batches.extend(torch.tensor(batch_X).float())


In [29]:
X_train = torch.stack(X_train_batches)
y_train = torch.stack(y_train_batches)
X_test = torch.stack(X_test_batches)
y_test = torch.stack(y_test_batches)

In [30]:
print(X_train.shape) # Samples, Months ,Features

torch.Size([30536, 8, 15])


# LSTM Simple

In [34]:
class AirModel(nn.Module):
    def __init__(self):
        super().__init__()
        print(input_size)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=128, num_layers=4, batch_first=True)
        self.linear = nn.Linear(128, 1)

    def forward(self, x):
        """
        x shape: batch_size x sequence_length (look_back) x num_features
        """
        x, _ = self.lstm(x)
        x = self.linear(x) # batch_size x sequence_length x 1
        x = x[:, -1, :]
        x = x.T.flatten()

        return x

In [36]:
# Define custom weights for each target value
def get_weights(y_batch):
    weights = torch.zeros_like(y_batch, dtype=torch.float)
    weights[y_batch == 0] = 0.1
    weights[y_batch == 1] = 0.2
    weights[y_batch == 2] = 0.25
    weights[y_batch >= 3] = 0.5
    return weights

model = AirModel()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.MSELoss(reduction='none')  # Using MSE loss with 'none' reduction
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=30)

n_epochs = 3
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)

        # Calculate custom weights for each sample in the batch
        weights = get_weights(y_batch)

        optimizer.zero_grad()
        weighted_loss = torch.mean(weights * loss)  # Apply weights to the loss
        weighted_loss.backward()
        optimizer.step()

    # Validation
    if epoch % 1 != 0:
        continue
    model.eval()
    with torch.no_grad():
        print("weighted loss", weighted_loss)
        y_pred = model(X_train)
        train_loss = loss_fn(y_pred, y_train)
        train_rmse = np.sqrt(torch.mean(train_loss))
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        test_rmse = np.sqrt(torch.mean(test_loss))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch+1, train_rmse, test_rmse))


weighted loss tensor(0.1133, grad_fn=<MeanBackward0>)
Epoch 1: train RMSE 1.4084, test RMSE 1.0639
weighted loss tensor(1.6323, grad_fn=<MeanBackward0>)
Epoch 2: train RMSE 1.4961, test RMSE 1.2208
weighted loss tensor(1.0123, grad_fn=<MeanBackward0>)
Epoch 3: train RMSE 1.4610, test RMSE 1.1608


In [37]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
print(len(y_pred))
print("Pred round:\n",y_pred.T.round()[500:550], "\n")

#print(y_pred[550:600], "\n")

print("Test:\n",y_test[500:550], "\n")

print("correct", torch.sum(y_pred.round() == y_test.round()))
print("all", len(y_pred))
print("zeros", torch.sum(y_test.round() == 0))

print(torch.sum((y_pred.round() == 1) & (y_pred.round() == y_test.round())))
print(torch.sum((y_pred.round() == 1)))

print(torch.sum((y_test.round() == 1)))

1388
Pred round:
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]) 

Test:
 tensor([0., 0., 0., 0., 0., 0., 1., 1., 2., 4., 0., 3., 1., 1., 1., 3., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) 

correct tensor(116)
all 1388
zeros tensor(1202)
tensor(116)
tensor(1388)
tensor(116)


<ipython-input-37-a246109bfbba>:5: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3614.)
  print("Pred round:\n",y_pred.T.round()[500:550], "\n")


In [ ]:
print(torch.sum((y_pred.round() >= 1) & (y_pred.round() == y_test.round())))

tensor(69)


In [ ]:
print(len(y_test))
print((y_test == 0).sum())
print((y_test == 1).sum())
print((y_test == 2).sum())
print((y_test == 3).sum())
print((y_test == 4).sum())
print((y_test == 5).sum())
print((y_test == 6).sum())
print((y_test == 7).sum())

2082
tensor(1797)
tensor(190)
tensor(65)
tensor(23)
tensor(7)
tensor(0)
tensor(0)
tensor(0)


In [ ]:
print(y_pred.round().unique())
print(y_pred.unique())

tensor([-0., 1., 2., 3.])
tensor([-4.3150e-02, -3.8366e-02, -3.7188e-02, -3.6721e-02, -1.9346e-02,
         1.6920e-03,  3.0016e-02,  4.0333e-02,  4.3748e-02,  5.8640e-02,
         6.8418e-02,  7.6406e-02,  8.1269e-02,  8.4553e-02,  9.0865e-02,
         9.4020e-02,  9.8343e-02,  1.0699e-01,  1.0708e-01,  1.0810e-01,
         1.1209e-01,  1.1800e-01,  1.1931e-01,  1.1936e-01,  1.2760e-01,
         1.2890e-01,  1.3148e-01,  1.4025e-01,  1.4196e-01,  1.4227e-01,
         1.4475e-01,  1.4796e-01,  1.4803e-01,  1.5079e-01,  1.6175e-01,
         1.6288e-01,  1.6591e-01,  1.7043e-01,  1.7267e-01,  1.8344e-01,
         1.9228e-01,  1.9714e-01,  1.9731e-01,  2.0126e-01,  2.0397e-01,
         2.0582e-01,  2.1532e-01,  2.1694e-01,  2.1815e-01,  2.2199e-01,
         2.2321e-01,  2.3637e-01,  2.3675e-01,  2.4794e-01,  2.4994e-01,
         2.5192e-01,  2.6014e-01,  2.6206e-01,  2.6726e-01,  2.6776e-01,
         2.7067e-01,  2.7726e-01,  2.9201e-01,  2.9223e-01,  2.9294e-01,
         2.9362e-01,  2.9

# TARPML

In [65]:
class TrafficAccidentLSTM(nn.Module):
    def __init__(self, input_size_lstm, hidden_size_lstm, num_layers_lstm, input_size_fc, hidden_size_fc, num_layers_fc, output_size, dropout_rate):
        super(TrafficAccidentLSTM, self).__init__()

        self.lstm = nn.LSTM(input_size=input_size_lstm, hidden_size=hidden_size_lstm, num_layers=num_layers_lstm, batch_first=True)

        self.fc_layers = nn.ModuleList()
        self.fc_layers.append(nn.Linear(input_size_fc + hidden_size_lstm, hidden_size_fc))
        for _ in range(num_layers_fc - 1):
            self.fc_layers.append(nn.Linear(hidden_size_fc, hidden_size_fc))

        self.output_layer = nn.Linear(hidden_size_fc, output_size)

        self.dropout = nn.Dropout(dropout_rate)

        self.relu = nn.ReLU()

    def forward(self, input_seq_lstm, input_fc):
        lstm_out, _ = self.lstm(input_seq_lstm)

        #lstm_out = lstm_out[:, -1, :]  # Taking the output of the last timestep

        print(lstm_out.shape)
        print(input_fc.shape)

        combined_input = torch.cat((lstm_out, input_fc), dim=2)

        print(combined_input.shape)

        for fc_layer in self.fc_layers:
            combined_input = self.dropout(combined_input)
            combined_input = self.relu(fc_layer(combined_input))

        output = self.output_layer(combined_input)

        return output

In [66]:
# Define custom weights for each target value
def get_weights(y_batch):
    weights = torch.zeros_like(y_batch, dtype=torch.float)
    weights[y_batch == 0] = 0.1
    weights[y_batch == 1] = 0.2
    weights[y_batch == 2] = 0.25
    weights[y_batch >= 3] = 0.5
    return weights

input_size_lstm = 4
hidden_size_lstm = 20
num_layers_lstm = 4

input_size_fc = 15-4
hidden_size_fc = 10
num_layers_fc = 3

output_size = 1  # Predicted traffic accident risk (frequency)

dropout_rate = 0.5

model = TrafficAccidentLSTM(input_size_lstm, hidden_size_lstm, num_layers_lstm, input_size_fc, hidden_size_fc, num_layers_fc, output_size, dropout_rate)

optimizer = optim.Adam(model.parameters(), lr=0.0001)
loss_fn = nn.MSELoss(reduction='none')  # Using MSE loss with 'none' reduction
loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=30)

n_epochs = 3
for epoch in range(n_epochs):
    model.train()
    for X_batch, y_batch in loader:
        y_pred = model(X_batch[:,:,:10], X_batch[:,:,-5:])
        loss = loss_fn(y_pred, y_batch)

        # Calculate custom weights for each sample in the batch
        weights = get_weights(y_batch)

        optimizer.zero_grad()
        weighted_loss = torch.mean(weights * loss)  # Apply weights to the loss
        weighted_loss.backward()
        optimizer.step()

    # Validation
    if epoch % 1 != 0:
        continue
    model.eval()
    with torch.no_grad():
        print("weighted loss", weighted_loss)
        y_pred = model(X_train)
        train_loss = loss_fn(y_pred, y_train)
        train_rmse = np.sqrt(torch.mean(train_loss))
        y_pred = model(X_test)
        test_loss = loss_fn(y_pred, y_test)
        test_rmse = np.sqrt(torch.mean(test_loss))
    print("Epoch %d: train RMSE %.4f, test RMSE %.4f" % (epoch+1, train_rmse, test_rmse))


torch.Size([30, 8, 20])
torch.Size([30, 8, 5])
torch.Size([30, 8, 25])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (240x25 and 31x10)

In [ ]:
model.eval()
with torch.no_grad():
    y_pred = model(X_test)
print(len(y_pred))
print("Pred round:\n",y_pred.T.round()[500:550], "\n")

#print(y_pred[550:600], "\n")

print("Test:\n",y_test[500:550], "\n")

print("correct", torch.sum(y_pred.round() == y_test.round()))
print("all", len(y_pred))
print("zeros", torch.sum(y_test.round() == 0))

print(torch.sum((y_pred.round() == 1) & (y_pred.round() == y_test.round())))
print(torch.sum((y_pred.round() == 1)))

print(torch.sum((y_test.round() == 1)))